# Manipulação de Cores e Histogramas no OpenCV

## 1. Configuração Inicial
Nesta aula, vamos aprofundar no conceito de **Espaços de Cores** e análise de imagens.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

def mostrar_imagem(imagem, titulo="Imagem"):
    # Se a imagem for P&B (2 dimensões), não precisa converter cores, mas precisa do cmap='gray'
    if len(imagem.shape) == 2:
        plt.figure(figsize=(8, 6))
        plt.imshow(imagem, cmap='gray')
        plt.title(titulo)
        plt.axis('off')
        plt.show()
    else:
        # Converte BGR para RGB
        imagem_rgb = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(8, 6))
        plt.imshow(imagem_rgb)
        plt.title(titulo)
        plt.axis('off')
        plt.show()

In [ ]:
# Carregando uma imagem colorida da pasta de amostras
caminho_img = r"../amostras/image-pose.jpg"
img = cv2.imread(caminho_img)

if img is None:
    print("Erro ao carregar imagem.")
else:
    mostrar_imagem(img, "Imagem Original")

# 2. Separação de Canais (Split)

Uma imagem colorida digital é formada pela sobreposição de 3 canais: **Blue, Green, Red (BGR)**.
Podemos separar esses canais para analisá-los individualmente.

In [ ]:
b, g, r = cv2.split(img)

# Visualizando os canais como escala de cinza (intensidade)
plt.figure(figsize=(12, 4))
plt.subplot(1,3,1); plt.imshow(b, cmap='gray'); plt.title("Canal Azul (Intensidade)")
plt.subplot(1,3,2); plt.imshow(g, cmap='gray'); plt.title("Canal Verde (Intensidade)")
plt.subplot(1,3,3); plt.imshow(r, cmap='gray'); plt.title("Canal Vermelho (Intensidade)")
plt.show()

### 2.1 Visualização Realista dos Canais
Ver o canal em cinza é abstrato. Vamos criar uma visualização onde o **Canal Vermelho aparece Vermelho**.
Para fazer isso, criamos uma imagem onde os outros dois canais são zero (preto).

In [ ]:
zeros = np.zeros(img.shape[:2], dtype="uint8")

# Canal Azul Real: [B, 0, 0]
img_b = cv2.merge([b, zeros, zeros])

# Canal Verde Real: [0, G, 0]
img_g = cv2.merge([zeros, g, zeros])

# Canal Vermelho Real: [0, 0, R]
img_r = cv2.merge([zeros, zeros, r])

plt.figure(figsize=(12, 4))
plt.subplot(1,3,1); plt.imshow(cv2.cvtColor(img_b, cv2.COLOR_BGR2RGB)); plt.title("Canal Azul")
plt.subplot(1,3,2); plt.imshow(cv2.cvtColor(img_g, cv2.COLOR_BGR2RGB)); plt.title("Canal Verde")
plt.subplot(1,3,3); plt.imshow(cv2.cvtColor(img_r, cv2.COLOR_BGR2RGB)); plt.title("Canal Vermelho")
plt.show()

# 3. Outros Espaços de Cor

O RGB não é o único jeito de representar cores. O **HSV** (Hue, Saturation, Value) é muito usado porque separa a **Cor (Matiz)** da **Luz (Valor)**.

- **H (Hue)**: É a cor pura (0-179 no OpenCV).
- **S (Saturation)**: O quão "viva" é a cor.
- **V (Value)**: Brilho.

In [ ]:
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(hsv)

mostrar_imagem(h, "Canal Hue (Matiz)")

# Convertendo para Grayscale (apenas Iluminação)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
mostrar_imagem(gray, "Grayscale (Escala de Cinza)")

# 4. Histogramas

O histograma é um gráfico que mostra a frequência de cada tom de cor na imagem (de 0 a 255).
Ele nos diz se a imagem está muito clara, muito escura, ou equilibrada.

In [ ]:
plt.figure(figsize=(10, 5))
plt.title("Histograma de Cores RGB")
plt.xlabel("Intensidade (Bins)")
plt.ylabel("# de Pixels")

cores = ('b', 'g', 'r')
# Itera sobre as cores e desenha a linha correspondente
for i, cor in enumerate(cores):
    # calcHist(imagens, canais, mascara, bins, range)
    hist = cv2.calcHist([img], [i], None, [256], [0, 256])
    plt.plot(hist, color=cor)
    plt.xlim([0, 256])

plt.show()

# 5. Binarização (Thresholding)

A binarização transforma a imagem em apenas preto e branco puro (0 e 255). É crucial para OCR e detecção de formas.

### 5.1 Limiar Fixo
Definimos um valor (ex: 127). Tudo acima vira branco, tudo abaixo vira preto.

In [ ]:
# Suaviza a imagem antes para reduzir ruído
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

# (imagem, limiar logic, valor maximo, tipo)
_, thresh_simples = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY)

mostrar_imagem(thresh_simples, "Limiar Simples (127)")

### 5.2 Limiar Adaptativo
O computador calcula o limiar ideal para pequenas regiões da imagem. Funciona muito melhor quando a iluminação não é uniforme.

In [ ]:
thresh_adapt = cv2.adaptiveThreshold(
    blurred, 255, 
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
    cv2.THRESH_BINARY, 
    11, 2 # Tamanho do bloco, constante subtraída
)

mostrar_imagem(thresh_adapt, "Limiar Adaptativo")